In [26]:
import csv
import math
import random

def loadCsv(filename):
    lines = csv.reader(open('diabetes.csv'))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset


In [27]:
def splitDataset(dataset, splitRatio):
    trainsize = int(len(dataset)*splitRatio)
    trainset = []
    copy = list(dataset)
    while len(trainset) < trainsize:
        index = random.randrange(len(copy))
        trainset.append(copy.pop(index))
    return [trainset, copy]

In [56]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [46]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [47]:
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [48]:
def summarize(dataset):
    summaries = [(mean(attribute),stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [49]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classvalue, instances in separated.items():
        summaries[classvalue] = summarize(instances)
    return summaries

In [50]:
def calculateProbability(x,mean,stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1/(math.sqrt(2*math.pi)*stdev))*exponent

In [62]:
def calculateClassProbabilities(summaries, inputvector):
    probabilities = {}
    for classvalue, classsummaries in summaries.items():
        probabilities[classvalue] = 1
        for i in range(len(classsummaries)):
            mean, stdev = classsummaries[i]
            x = inputvector[i]
            probabilities[classvalue]*= calculateProbability(x,mean,stdev)
        return probabilities

In [63]:
def predict(summaries,inputvector):
    probabilities = calculateClassProbabilities(summaries,inputvector)
    bestLabel, bestProb = None, -1
    for classvalue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classvalue
        return bestLabel

In [64]:
def getPredictions(summaries,testset):
    predictions = []
    for i in range(len(testset)):
        result = predict(summaries, testset[i])
        predictions.append(result)
    return predictions

In [65]:
def getAccuracy(testset, predictions):
    correct = 0
    for x in range(len(testset)):
        if testset[x][-1] == predictions[x]:
            correct+=1
    return (correct/float(len(testset)))*100.0

In [66]:
def main():
    filename = 'diabetes.csv'
    splitRatio = 0.67
    dataset = loadCsv(filename)
    trainingSet,testset = splitDataset(dataset,splitRatio)
    print(f'split {len(dataset)} rows into train = {len(trainingSet)} rows and test = {len(testset)} rows')
    # Preparing Model
    summaries = summarizeByClass(trainingSet)
    # Test Model
    predictions = getPredictions(summaries,testset)
    accuracy = getAccuracy(testset,predictions)
    print(f'Accuracy: {accuracy}')
    
main()

split 768 rows into train = 514 rows and test = 254 rows
Accuracy: 67.32283464566929
